In [41]:
# 데이터 후처리
# 기존 111번까지는 phenotype / score / z_score / p_value 파일 존재
# 이후 score scaling
# 이후 p_value 0.001 ~ 1
# 이후 "수정_id2" 파일을 통해 phenotype 컬럼을 기준으로 일치하면 umlsid, hpoid 컬럼 추가
# 이후 "phenotype mappingv4" 파일을 통해 umlis 컬럼을 기준으로 일치하면 group, groupid 추가

import numpy as np
import pandas as pd
import configparser
import psycopg2
import os
import ast
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
from collections import defaultdict
from statistics import harmonic_mean
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

number=7 #수정 필요

with open("result/{}차/output_dict.pickle".format(number),"rb") as fi:
    output_dict = pickle.load(fi)
    
# validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number))
inferred_phenotype2 = pd.read_csv('result/{}차/{}_network_score.csv'.format(number,number))

phenotype_rwr_score_dict = pd.Series(inferred_phenotype2.Network_score.values, index=inferred_phenotype2.Phenotype).to_dict()

temp=list(phenotype_rwr_score_dict.keys())
for i in temp:
    if np.isnan(phenotype_rwr_score_dict[i])==True:
        phenotype_rwr_score_dict.pop(i)

Phenotype_score_Phenotype=list(phenotype_rwr_score_dict.keys())
Phenotype_score_Score=list(phenotype_rwr_score_dict.values())

temp2=list(output_dict.keys())
for i in temp2:
    if np.isnan(output_dict[i][0])==True:
        output_dict.pop(i)
        
P_value_Phenotype=list(output_dict.keys())
P_value_Score=list(output_dict.values())

count=0
z_score_list=[]
outlier_list=[] #제거가능
P_value_list=[]
rank_number=0
result_phenotype=[]
result_phenotype_score=[]

for i in range(len(P_value_Score)):
    Numberlist=P_value_Score[i].copy()
    tmp1=P_value_Phenotype[i]
    tmp2=Phenotype_score_Phenotype.index(tmp1)
    tmp3=Phenotype_score_Score[tmp2]
    Numberlist.append(tmp3)
    result_phenotype.append(tmp1)
    result_phenotype_score.append(tmp3)
    
    # 평균 계산
    mean = calculate_mean(Numberlist)
    # 표준편차 계산
    std_deviation = calculate_standard_deviation(Numberlist)

    z_score=(tmp3-mean)/std_deviation
    z_score_list.append(z_score)
    if z_score >=-2.58 and z_score<=2.58: #제거가능
        outlier=1
        count=count+1
    else:
        outlier=0 
    outlier_list.append(outlier) #제거가능

    Numberlist.sort(reverse=True)

    rank=Numberlist.index(tmp3) #순위찾기
    if rank<5:
        rank_number=rank_number+1
    P_value_list.append(rank)
#     print("phe",tmp1)
#     print("z",z_score)
#     print("p",rank)

# postgresql DB 연동
config = configparser.ConfigParser()
config.read('db_config.ini')
conn=psycopg2.connect(host="168.131.30.66", dbname="coconut",user="dbuser",password="jnudl1") #데이터이름변경
conn_cur = conn.cursor()

#------------------------

Phenotype_score = pd.DataFrame({
    'Phenotype': result_phenotype,
    'Network_score': result_phenotype_score,
    'z-score': z_score_list,
    'P-value': P_value_list,
})

# print(Phenotype_score)

Phenotype_score=Phenotype_score.sort_values(by='P-value',ascending=True)

#     # Compound ID에 해당하는 inferred phenotype.csv 생성
# if not os.path.exists('compound 결과'):
#     os.makedirs('compound 결과')
Phenotype_score.to_csv('result/{}차/{}_검증.csv'.format(number,number),index=False, encoding='UTF-8')

#------------------------------------
# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = Phenotype_score['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
Phenotype_score['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

Phenotype_score['P-value'] = Phenotype_score['P-value'].apply(adjust_pvalue)


#------------------------------------
#umls db에서 umls id와 phenotype score 파일을 합치는 코드

file_2= pd.read_csv("Data/수정_Id2.csv")

# file과 file_2를 'Phenotype'과 'phenotype' 컬럼을 기준으로 결합합니다.
# 이때 file_2의 'umlsid' 컬럼만 선택하여 추가합니다.
file_3 = pd.merge(Phenotype_score, file_2[['umlsid', 'phenotype','hpoid']], left_on='Phenotype', right_on='phenotype', how='left')

# 중복된 'phenotype' 컬럼을 제거합니다.
file_3.drop(columns=['phenotype'], inplace=True)
Phenotype_score=file_3
# print(Phenotype_score)
#------------------------------------
# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('Data/phenotype mappingv4.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
grouped_mapping_df = mapping_df.groupby('UMLSID')['Group'].agg(lambda x: ','.join(x)).reset_index()

# Phenotype_score와 grouped_mapping_df를 병합합니다.
final_df = pd.merge(Phenotype_score, grouped_mapping_df, left_on='umlsid', right_on='UMLSID', how='left')

# 필요없는 UMLSID 컬럼을 제거합니다.
final_df.drop(columns=['UMLSID'], inplace=True)

cols_to_drop = [col for col in final_df.columns if col.startswith("Unnamed")]
final_df = final_df.drop(columns=cols_to_drop)

final_df.to_csv('result/{}차/{}_검증_with_scaling.csv'.format(number,number), index=False) 

In [ ]:
# number=1
# text=["adenine", "canavanin", "malate", "isoleucine", "phenylalanine", "tyrosine", "isocitrate", "glutamine", "tryptophan","L-malate", 
#       "theogallin", "trans-aconitate", "cis-aconitate", "threonate", "benzoic acid", "choline", "citric acid", "coumarin", "phloroglucinol", "gallic acid", 
#       "citrullin", "pipecolic acid", "vitamin B", "pantothenic acid", "DL-valine", "vanillin", "theophylline", "caffeine", "protocatechuic aldehyde", "adenosine", 
#       "EGCG", "gallocatechin", "N-acetylglutamate", "loliolide", "procyanidin B2", "3H-proline", "dihydromyricetin", "beta-alanine betaine", "epicatechin gallate (ECG", "5'-methylthioadenosine", 
#       "epiafzelechin", "ferulic acid", "p-coumaric acid", "caffeic acid", "chlorogenic acid", "quercetin", "alpha-isopropylmalate", "neochlorogenic acid", "rutin", "hyperoside", 
#       "4-p-coumaroylquinic acid", "astragalin", "quercetin 7-O-glucoside", "p-coumaroylquinic acid", "procyanidin B1"]

# number=2
# text=["gamma-aminobutyric acid (GABA", "canavanin", "sucrose", "folate", "guanosine", "vanillic acid", "ononin", 
#       "formononetin", "calycosin", "sissotrin", "afrormosin", "pratensein"]

# number=3
# text=["protocatechuic acid", "gamma-aminobutyric acid (GABA", "choline", "citric acid", "malate", "asparagine", 
#       "quinic acid", "maltol", "nodakenetin", "nodakenin", "columbianetin", "decursin", "ferulic acid", "caffeic acid",
#       "aegelinol", "cis-chlorogenic acid", "chlorogenic acid", "esculetin"]

# number=4
# text=["piperine", "piperanine", "piperlonguminine", "retrofractamide B", "dehydropipernonaline", "Methyl piperate", "pipernonaline", "Retrofractamide A", "retrofractamide C"]

# number=5
# text=["apocynin", "resacetophenone", "4-hydroxyacetophenone", "2,5-dihydroxyacetophenone", "physcion"]

# number=6
# text=["protocatechuic acid", "p-hydroxybenzaldehyde", "choline", "gluconate", "myo-inositol", "thymine", "uracil", "vanillin", "vanillic acid", "protocatechuic aldehyde", "trans-aconitate", "ferulic acid", "caffeic acid", "chlorogenic acid", "linoleic acid", "neochlorogenic acid", "xanthatin", "cynarin", "cynarine"]

# number=7
# text=["atractylenolide III", "vanillic acid", "esculetin", "elemicin", "adenosine"]


In [ ]:
#HPO gene_phenotype 데이터와 PPI network 내에서 entrez_list들의 교집합을 phenotype / entrez_id 형식으로 변환

# herb 최종
from tqdm import tqdm_notebook
import numpy as np
import pickle
import pandas as pd
import configparser
import psycopg2
import os
import ast
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
import copy
from collections import defaultdict
from statistics import harmonic_mean

start = time.time()

def calculate_mean(numbers):
    return sum(numbers) / len(numbers)

def calculate_standard_deviation(numbers):
    return statistics.stdev(numbers)

def combine_lists(full_list):
    combined = defaultdict(list)
    
    # 모든 리스트에 대한 처리
    for sublist in full_list:
        for item in sublist:
            entrez_id, rwr_score = item
            combined[entrez_id].append(rwr_score)
    
    # 조화평균 및 결과 리스트 만들기
    result = []
    for entrez_id, scores in combined.items():
        if len(scores) == 1:
            result.append([entrez_id, scores[0]])  # 값이 하나면 그대로 추가
        else:
            # 조화평균 계산
            harmonic_mean_score = harmonic_mean(scores)
            result.append([entrez_id, harmonic_mean_score])
    
    return result

def compound_target(compoundid):

    conn_cur.execute(f"with tmp1 as (select cg.geneid from compound_gene cg where (cg.compid = '{compoundid}' and cg.relationsign in ('activate','inhibit')))" 
                    f"select g.entrezid from gene g,tmp1 where g.geneid=tmp1.geneid;")
    compound_gene1=conn_cur.fetchall()

    compound_gene=set(compound_gene1)
    compound_gene=list(compound_gene)
    
    # Compound-target gene List 생성
    target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            target_gene_list.append(interaction_gene)        
        else:
            continue

    # restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in target_gene_list:
        node_ = node_index.get(entrez) 
        target_node.append(node_)

    conn_cur.execute(f"with tmp2 as (select cg.geneid from compound_gene cg where cg.compid = '{compoundid}' and relationsign='associate')" 
                    f"select g.entrezid from gene g,tmp2 where g.geneid=tmp2.geneid;")

    compound_gene2=conn_cur.fetchall()

    compound_gene=set(compound_gene2)
    compound_gene=list(compound_gene)

    indirected_target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            indirected_target_gene_list.append(interaction_gene)        
        else:
            continue      

    num1=len(target_gene_list) #target gene list 갯수
    directed_target_gene_list.append(num1)
    num2=len(indirected_target_gene_list) #indirected target gene list 갯수 
    indirected_target_gene_list_count.append(num2)
    
    #indirect nodes 정의
    indirect_nodes= []
    for entrez in indirected_target_gene_list:
        node_ = node_index.get(entrez) 
        indirect_nodes.append(node_)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 

    # RWR 실행
    rwr = RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    
    var_name1 = f'{compoundid}_num1'
    var_name2 = f'{compoundid}_num2'

    # 전역 변수 설정
    globals()[var_name1] = num1
    globals()[var_name2] = num2
    
    return rwr_mapping_entrez

def rwr_compound_target(count,entrez_list):
    
    num1=directed_target_gene_list[count]
    num2=indirected_target_gene_list_count[count]
    
    random_interaction_gene1 = random.sample(entrez_list, num1)
    remaining_entrez_list = [gene for gene in entrez_list if gene not in random_interaction_gene1]
    random_interaction_gene2 = random.sample(remaining_entrez_list, num2)
    # restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in random_interaction_gene1:
        node_ = node_index.get(entrez)
        target_node.append(node_)

    #indirect nodes 정의
    indirect_nodes= []
    for entrez in random_interaction_gene2:
        node_ = node_index.get(entrez)
        indirect_nodes.append(node_)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 
    
    # RWR 실행
    rwr=RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    return rwr_mapping_entrez

def create_ppi_network(network_data):
    network_data = network_data.astype({'Entrez Gene Interactor A': str,
                                        'Entrez Gene Interactor B': str})

    # symbol list를 생성하고 unique 값만 가지도록 생성
    symbolA = network_data.loc[:, 'Entrez Gene Interactor A'].to_list()
    symbolB = network_data.loc[:, 'Entrez Gene Interactor B'].to_list()

    symbol_list = symbolA + symbolB 
    symbol_list = set(symbol_list) 
    symbol_list = list(symbol_list)

    # 모든 symbol에 index를 딕셔너리에서 부여
    node_index = {}
    for i in range(len(symbol_list)):
        node_index[symbol_list[i]] = i

    # index로 node_list 생성
    node_list = node_index.values()
    
    # index로 edge_list 생성
    edge_list = network_data[['Entrez Gene Interactor A', 'Entrez Gene Interactor B']].values.tolist()  #
    for i in range(len(edge_list)):
        edge_list[i][0] = node_index.get(edge_list[i][0])
        edge_list[i][1] = node_index.get(edge_list[i][1])
    
    # 무방향 그래프 오브젝트 생성
    G = nx.Graph()

    # 그래프에 노드 추가
    G.add_nodes_from(node_list)

    # 그래프에 엣지 추가
    G.add_edges_from(edge_list)

    print(f'Number of created nodes: {G.number_of_nodes()}')
    print(f'Number of created edges: {G.number_of_edges()}')

    return G, node_index

def RWR_initializing(G, seed_nodes, indirect_target_nodes, is_weighted=False):
    norm_A = np.zeros(shape=(len(G), len(G)))
    if is_weighted:  
        for i, neighbor_dict in G.adjacency():
            for j, v in neighbor_dict.items():
                norm_A[i][j] = v.get("weight", 1 / len(neighbor_dict))
    else:  
        for i, neighbor_dict in G.adjacency():  
            for j, v in neighbor_dict.items():  
                norm_A[i][j] = 1 / len(neighbor_dict)
    
    # seed nodes와 indirect target nodes를 모두 포함한 딕셔너리 생성
    personalization = {node: 1 for node in seed_nodes}
    for node in indirect_target_nodes:
        personalization[node] = 0.3  
        
    r_0 = np.array([personalization.get(n, 0) for n in range(len(G))]) 
    r_c = np.repeat(1 / len(G), len(G)) 
    return {"norm_A": norm_A, "r_0": r_0, "r_c": r_c, "N": len(G)}

def RWR(initialized, prob=0.85, max_iter=100, tol=1.0e-6):
    if initialized is None:
        raise ValueError('initialized information must be required')

    norm_A = initialized['norm_A']  # 인접행렬 A (정규화된)
    r_0 = initialized['r_0']        # 시작 노드
    r_c = initialized['r_c']        # 가중치 행렬
    N = initialized['N']            # 노드의 개수
    for iteration in range(max_iter):
        r_prev = r_c
        r_c = prob * r_c @ norm_A + (1 - prob) * r_0  # RWR algorithm
        err = np.absolute(r_c - r_prev).sum()
        if err < N * tol:
#             print(f'RWR iteration = {iteration + 1}')
#             print('Converged')
            return r_c
#         else:
#             print(f'RWR iteration = {iteration + 1}, Iteration until convergence ...')
#             print(f'{err} -> {N * tol}')
    return "NotConverged"

# BIOGRID-Homo_sapiens Data 기반 PPI Network 구축
network_data = pd.read_csv("/data/home/sss2061/식약처/Data/BIOGRID-ORGANISM-Homo_sapiens.tab3.txt", sep="\t", encoding="cp949")  
G, node_index = create_ppi_network(network_data=network_data)  
entrez_list = list(node_index.keys())
entrez_list.remove('-')
gene_phenotype_file = '/data/home/sss2061/식약처보고서/서울과기대/Data/HPO_gene_phenotype.csv'
gene_phenotype_df= pd.read_csv(gene_phenotype_file, encoding='UTF-8', converters={'Phenotype': ast.literal_eval})
# 각 phenotype에 대한 Entrez ID들의 리스트를 만들기 위한 빈 딕셔너리
phenotype_entrez_dict = {}

#리스트 내 요소 문자형 컴마 -> 정수형으로 변경
entrez_list = [int(item) for item in entrez_list]

# 각 행에 대해 반복
for index, row in gene_phenotype_df.iterrows():
    entrez_id = row['Entrez ID']
    if entrez_id in entrez_list:
        for phenotype in row['Phenotype']:
            if phenotype in phenotype_entrez_dict:
                phenotype_entrez_dict[phenotype].add(entrez_id)
            else:
                phenotype_entrez_dict[phenotype] = {entrez_id}

# 딕셔너리에서 데이터프레임으로 변환
final_df = pd.DataFrame([(key, list(values)) for key, values in phenotype_entrez_dict.items()], 
                        columns=['Phenotype', 'Entrez ID'])
final_df.to_csv('phenotype_entrezid.csv', index=False)

In [11]:
# 데이터 후처리
# 기존 111번까지는 phenotype / score / z_score / p_value 파일 존재
# 이후 score scaling
# 이후 p_value 0.001 ~ 1
# 이후 "수정_id2" 파일을 통해 phenotype 컬럼을 기준으로 일치하면 umlsid, hpoid 컬럼 추가
# 이후 "phenotype mappingv4" 파일을 통해 umlis 컬럼을 기준으로 일치하면 group, groupid 추가
# groupid와 group을 같은 행이 아닌 다른 행에 추가할때 사용하는 코드

import numpy as np
import pandas as pd
import configparser
import psycopg2
import os
import ast
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
from collections import defaultdict
from statistics import harmonic_mean
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle

number=5 #수정 필요

with open("result/{}차/output_dict.pickle".format(number),"rb") as fi:
    output_dict = pickle.load(fi)
    
# validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number))
inferred_phenotype2 = pd.read_csv('result/{}차/{}_network_score.csv'.format(number,number))

phenotype_rwr_score_dict = pd.Series(inferred_phenotype2.Network_score.values, index=inferred_phenotype2.Phenotype).to_dict()

temp=list(phenotype_rwr_score_dict.keys())
for i in temp:
    if np.isnan(phenotype_rwr_score_dict[i])==True:
        phenotype_rwr_score_dict.pop(i)

Phenotype_score_Phenotype=list(phenotype_rwr_score_dict.keys())
Phenotype_score_Score=list(phenotype_rwr_score_dict.values())

temp2=list(output_dict.keys())
for i in temp2:
    if np.isnan(output_dict[i][0])==True:
        output_dict.pop(i)
        
P_value_Phenotype=list(output_dict.keys())
P_value_Score=list(output_dict.values())

count=0
z_score_list=[]
outlier_list=[] #제거가능
P_value_list=[]
rank_number=0
result_phenotype=[]
result_phenotype_score=[]

for i in range(len(P_value_Score)):
    Numberlist=P_value_Score[i].copy()
    tmp1=P_value_Phenotype[i]
    tmp2=Phenotype_score_Phenotype.index(tmp1)
    tmp3=Phenotype_score_Score[tmp2]
    Numberlist.append(tmp3)
    result_phenotype.append(tmp1)
    result_phenotype_score.append(tmp3)
    
    # 평균 계산
    mean = calculate_mean(Numberlist)
    # 표준편차 계산
    std_deviation = calculate_standard_deviation(Numberlist)
    if std_deviation==0:    #제거
        z_score=0
    else:
        z_score=(tmp3-mean)/std_deviation

    z_score_list.append(z_score)
    if z_score >=-2.58 and z_score<=2.58: #제거가능
        outlier=1
        count=count+1
    else:
        outlier=0 
    outlier_list.append(outlier) #제거가능

    Numberlist.sort(reverse=True)

    rank=Numberlist.index(tmp3) #순위찾기
    if rank<5:
        rank_number=rank_number+1
    P_value_list.append(rank)
#     print("phe",tmp1)
#     print("z",z_score)
#     print("p",rank)

# postgresql DB 연동
config = configparser.ConfigParser()
config.read('db_config.ini')
conn=psycopg2.connect(host="168.131.30.66", dbname="coconut",user="dbuser",password="jnudl1") #데이터이름변경
conn_cur = conn.cursor()

#------------------------

Phenotype_score = pd.DataFrame({
    'Phenotype': result_phenotype,
    'Network_score': result_phenotype_score,
    'z-score': z_score_list,
    'P-value': P_value_list,
})

# print(Phenotype_score)

Phenotype_score=Phenotype_score.sort_values(by='P-value',ascending=True)

#     # Compound ID에 해당하는 inferred phenotype.csv 생성
# if not os.path.exists('compound 결과'):
#     os.makedirs('compound 결과')
Phenotype_score.to_csv('result/{}차/{}_검증.csv'.format(number,number),index=False, encoding='UTF-8')

#------------------------------------
# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = Phenotype_score['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
Phenotype_score['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

Phenotype_score['P-value'] = Phenotype_score['P-value'].apply(adjust_pvalue)


#------------------------------------
#umls db에서 umls id와 phenotype score 파일을 합치는 코드

file_2= pd.read_csv("Data/수정_Id2.csv")

# file과 file_2를 'Phenotype'과 'phenotype' 컬럼을 기준으로 결합합니다.
# 이때 file_2의 'umlsid' 컬럼만 선택하여 추가합니다.
file_3 = pd.merge(Phenotype_score, file_2[['umlsid', 'phenotype','hpoid']], left_on='Phenotype', right_on='phenotype', how='left')

# 중복된 'phenotype' 컬럼을 제거합니다.
file_3.drop(columns=['phenotype'], inplace=True)
Phenotype_score=file_3
# print(Phenotype_score)
#------------------------------------
# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('Data/phenotype mappingv4.xlsx', engine='openpyxl')

merged_df = pd.merge(Phenotype_score, mapping_df, left_on='umlsid', right_on='UMLSID', how='left')

# 필요없는 UMLSID 컬럼을 제거합니다.
merged_df.drop(columns=['UMLSID'], inplace=True)

cols_to_drop = [col for col in merged_df.columns if col.startswith("Unnamed")]
final_df = merged_df.drop(columns=cols_to_drop)

final_df.to_csv('result/{}차/{}_검증_with_scaling_버전2.csv'.format(number,number), index=False) 